# A08. Odds
This scrapes DraftKings moneylines, over/unders, and spreads 
- Type: Data
- Run Frequency: Pre-contest
- Sources:
    - Fantasy Labs
- Dates:
    - Created: 1/28/2024
    - Updated: 4/21/2024

In [8]:
def odds(date):
    # Convert to proper format
    date_dash = date[0:4] + "-" + date[4:6] + "-" + date[6:8]
    
    url = f'https://www.fantasylabs.com/api/sportevents/3/{date_dash}/vegas/?sportId=3&date={date_dash}'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    jsonData = requests.get(url, headers=headers).json()
    
    # Initialize an empty list to store extracted data
    data_list = []

    for game in jsonData:
        event_details = game['EventDetails']
        
        if event_details == {'Properties': {'Scores': []}, 'IsLive': False}:
            continue
        
        VisitorTeamShort = event_details['Properties']['VisitorTeamShort']
        HomeTeamShort = event_details['Properties']['HomeTeamShort']
        Spread = game['Spread']
        OU = game['OU']
        SpreadMoney1 = game['SpreadMoney1']
        SpreadMoney2 = game['SpreadMoney2']
        OuMoney1 = game['OuMoney1']
        OuMoney2 = game['OuMoney2']
        MLMoney1 = game['MLMoney1']
        MLMoney2 = game['MLMoney2']
        VisitorVegasRuns = event_details['Properties']['VisitorVegasRuns']
        HomeVegasRuns = event_details['Properties']['HomeVegasRuns']
        EventDateTime = game['EventDateTime']

        # Append extracted data to the list
        data_list.append({
            'VisitorTeamShort': VisitorTeamShort,
            'HomeTeamShort': HomeTeamShort,
            'Spread': Spread,
            'OU': OU,
            'SpreadMoney1': SpreadMoney1,
            'SpreadMoney2': SpreadMoney2,
            'OuMoney1': OuMoney1,
            'OuMoney2': OuMoney2,
            'MLMoney1': MLMoney1,
            'MLMoney2': MLMoney2,
            'VisitorVegasRuns': VisitorVegasRuns,
            'HomeVegasRuns': HomeVegasRuns,
            'EventDateTime': EventDateTime,
            'date': date
        })

    # Create a Pandas DataFrame from the list
    odds_df = pd.DataFrame(data_list)

    
    return odds_df

Calculate OuMoney2 from OuMoney1 <br>
Create a crosswalk of OuMoney1 to OuMoney2 values from 2022 when they were accurate, choosing the mode value for each OuMoney1 <br>

In [13]:
# %run "U1. Imports.ipynb"
def create_over_under_df():
    # Directory containing the CSV files
    directory = r'C:\Users\james\Documents\MLB\Database\A08. Odds'
    
    # List to store dataframes from CSV files
    dataframes = []
    
    # Loop through files in the directory
    for filename in os.listdir(directory):
        # Check if file starts with "Odds 2022" and ends with ".csv"
        if filename.startswith("Odds 2022") and filename.endswith(".csv"):
            # Read the CSV file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            # Append the dataframe to the list
            dataframes.append(df)
    
    # Concatenate all dataframes into one
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    # Group by 'OuMoney1' and calculate the mode of 'OuMoney2' within each group
    mode_df = combined_df.groupby('OuMoney1')['OuMoney2'].agg(lambda x: x.mode().iloc[0])
    
    # Convert the resulting series to a dataframe
    mode_df = mode_df.reset_index()
    
    # Print the resulting dataframe
    print(mode_df)

    # Save
    mode_df.to_csv(os.path.join(baseball_path, "Utilities", "OverUnder Converter.csv"), index=False)